## Introduction

Today we will be discussing and performing some more genome assembly, however this time you will each have different published datasets to work on.

## 5. Post-Assembly QC {data-link="5. Post-Assembly QC"}

Now that we have some assembled genomes using different datasets, we will explore them in some more detail. This should help you understand the strengths and weaknesses of the different data-types, and also how to assess different aspects of genome assembly quality.

Both Canu and Spades should have produced directories full of various files. Hifiasm should also produce output files, but for this program we need to do an additional step to generate a "fasta" file from the "gfa" file.

    awk '/^S/{print ">"$2;print $3}' hifi_ecoli.p_ctg.gfa > hifi_ecoli.p_ctg.fa

### 5.1. Assembly graphs and genome size.

We can visualise the assembly graphs for these three different datasets using the software Bandage. We use a specific file-type produced by some (but not all) genome assembly programs, known as a Graphical Fragment Assembly (.gfa) file. This format documents stretches of assembles sequence and also the relationship between contiguous runs, that may have multiple valid paths due to the presence of repetitive sequence or heterozygosity.

You can load bandage by:

1.  typing the command "Bandage" into the unix command prompt.

2.  A window should open.

3.  You can click File, then load graph, and you should be able to select a gfa file from one of your assemblies.

##### Figure 3: A visualisation of the gfa file produced by Spades using the short Illumina read data. We can see that there is a tangle of valid paths, and also that the total length is higher than the size estimated from our earlier k-mer analysis. There are long stretches of contiguous sequence, which is useful genomic data, but the unresolved regions of the graph may inhibit some types of analysis. We would refer to this as a draft genome.

![](images/Screenshot%202022-10-13%20at%2014.30.49.png){fig-align="center"}

##### Figure 4: In contrast, here is a visualisation of the "unitig" gfa file produced by canu using the nanopore data. We can see that, for the most part there is a single clean path (node), and that the total length is roughly the size we expect. After checking the final contig sequence file (fasta), the loop structure is resolved into a single circular genome.

![](images/Screenshot%202022-10-14%20at%2014.02.47.png){fig-align="center"}

##### Figure 5: Here is a visualisation of the gfa file produced by hifiasm using the PacBio HiFi dataset. We can see that this data is able to resolve the genome into a single closed loop.

![](images/Screenshot%202022-10-13%20at%2014.29.47.png){fig-align="center"}

### 5.2. Gene content

We can check the quality of an assembly, by checking whether "conserved" genes are present in our assemblies (see paper for details). Using the software BUSCO (Benchmarking Universal Single-Copy Orthologs), we can generate some information about whether genes are present or absent, fragmented across contigs, or whether they're present in the right number.

    ##Bash/command line code 
    busco -m genome -i /scratch/ecoli_hiseq_spades/scaffolds.fasta -o Hiseq_busco --auto-lineage-prok

### 5.3. k-mer content

Another way to check the quality of a genome assembly, without having to rely on genes, is to compare the k-mer content that we saw in the raw reads to the k-mers present in the finished genome assembly. We can use the k-mer analysis toolkit to do this.

    kat comp -t 16 -o SRR18106304_vs_spades SRR18106304.fastq ecoli_hiseq_spades/scaffolds.fasta

![](images/Screenshot%202022-10-24%20at%2016.42.40.png){fig-align="center"}

Here we can see the same distribution we observed earlier, however now we have colours added, which indicate how many times k-mers occur in the assembled genome. We can see here on the left hand side that many of the error k-mers are shown in black. This means the k-mers are not present in the assembly, which is good news. However, we can also see that some of these error k-mers are shown in red, which means there are some sequence errors still present in the draft assembly.

In comparison here is the same plot for the HiFi assembly.

![](images/Screenshot%202022-10-24%20at%2016.40.24.png){fig-align="center"}

Now we can see that there are much less k-mers in the left hand peak present in the assembly. although we can see that a small number of k-mers in the right hand peak contains a small amount of purple duplicated k-mers!

### 5.4. Quast report

Before we move on to some other checks we are going to run the program quast, which performs a range of checks for you. We can run this as indicated below. For the purposes of demonstration we will use a reference genome to check our data against, but for a novel genome assembly we would not have this data.

1.  First find an E. coli k-12 reference genome on NCBI (https://www.ncbi.nlm.nih.gov/), as shown below.

    ![](images/Screenshot%202022-10-26%20at%2010.26.34.png){fig-align="center"}

2.  Select one of the resulting genome assemblies.

3.  Now click on the link on the right hand side of the page labelled "FTP directory for RefSeq assembly"

4.  Here you can see a range of information including the genomic sequence files annotations and other associated statistics.

5.  Right click on the file with the suffix "\_genomic.fna.gz" and copy the link.

6.  Now in your unix terminal navigate to your home directory and paste the link after a "wget command", in the same format as below (make sure to paste in the correct link that you've copied).


```{=html}
<!-- -->
```

    wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_example/GCF_example_genomic.fna.gz

7.  Now you are ready to run quast with your three previously assembled genomes.


```{=html}
<!-- -->
```

    quast -r  GCF_000005845.2_ASM584v2_genomic.fna --circos --gene-finding --output-dir test_quast ./Hifi_ecoli.fa ./ecoli-oxford/ecoli.contigs.fasta ./ecoli_hiseq_spades/scaffolds.fasta  

8.  Have a look at the quast report by loading the html file.


```{=html}
<!-- -->
```

    firefox test_quast/report.html


In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_bio as dashbio
from dash import html, dcc

app = dash.Dash(__name__)

HOSTED_GENOME_DICT = [
    {'value': 'mm10', 'label': 'Mouse (GRCm38/mm10)'},
    {'value': 'rn6', 'label': 'Rat (RGCS 6.0/rn6)'},
    {'value': 'gorGor4', 'label': 'Gorilla (gorGor4.1/gorGor4)'},
    {'value': 'panTro4', 'label': 'Chimp (SAC 2.1.4/panTro4)'},
    {'value': 'panPan2', 'label': 'Bonobo (MPI-EVA panpan1.1/panPan2)'},
    {'value': 'canFam3', 'label': 'Dog (Broad CanFam3.1/canFam3)'},
    {'value': 'ce11', 'label': 'C. elegans (ce11)'}
]

app.layout = html.Div([
    dcc.Loading(id='default-igv-container'),
    html.Hr(),
    html.P('Select the genome to display below.'),
    dcc.Dropdown(
        id='default-igv-genome-select',
        options=HOSTED_GENOME_DICT,
        value='ce11'
    )
])


# Return the IGV component with the selected genome.
@app.callback(
    Output('default-igv-container', 'children'),
    Input('default-igv-genome-select', 'value')
)
def return_igv(genome):
    return html.Div([
        dashbio.Igv(
            id='default-igv',
            genome=genome,
            minimumBases=100,
        )
    ])

if __name__ == '__main__':
    app.run_server(debug=True)

## 6. More complex genomes {data-link="6. More complex genomes"}

So far we have only considered a haploid bacterial genome, with a very small overall size. In such simple genomes an assembly graphs may suggest multiple paths through a given sequence due to repeating sequence, but we know that there is likely only one biologically real path through the graph. In diploid (and polyploid) organisms, this becomes more complicated because there are more than one real biological paths through the graph. These alternate paths are due to the presence of sister chromatids, that are not identical to each other, due to the presence of genetic variation, or heterozygosity. A key factor in assembling high quality genome assemblies, is the "phasing" of the chromosomes. It can be difficult to distinguish whether a given heterozygous region belongs to one chromosome copy or the other. Variation found along the same chromosome is in "linkage" or can be said to have the same "phase". A perfect genome assembly would reconstruct the variation from each chromosome perfectly as found in the original organism, however in practice this can be difficult to achieve. The figures below should help to explain this problem and the potential solutions.

##### Figure 6: A common result from sequencing diploid genomes is the presence of redundant haplotypes in the final assembly, particularly for regions with have a high sequence divergence. Here we can see homozygous regions in orange, and a reconstructed reference contig which contains a mosaic of haplotypes from two different sister chromatids (indicated in yellow and blue).

![](images/Screenshot%202022-10-24%20at%2017.25.47.png){fig-align="center"}

##### Figure 7: One method of resolving heterozygosity in a genome sequencing project is to gather parental data and use this to partition the chromatids inherited by the F1 offspring. In this example, we generated long reads for an F1 individual and short Illumina reads for the parents. We were able to use the Illumina reads to separate the F1 PacBio reads into two groups for assembly.

![](images/Screenshot%202022-10-24%20at%2017.37.37.png){fig-align="center"}

##### Figure 7: Another way to utilise parental data is to use the k-mers during the construction of the assembly graph itself. Specifically, we can identify bubble structures that are likely to represent heterozygous regions and look for parent specific k-0mers in the alternate paths. Crucially, this approach requires high accuracy long reads such as HiFi data to work successfully.

![](images/Screenshot%202022-10-24%20at%2017.37.51.png){fig-align="center"}

###### 